In [1]:
%cd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab Notebooks/WSSS
%ls

/root
Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/WSSS
AdelaiDet/                   res38_cls.pth
ADL_notok.ipynb              SEAM/
AuxSegNet/                   SESS/
BANA/                        src/
BBAM/                        test_img2.jpg
BBAM_notok.ipynb             test_img.jpg
BESTIE/                      TokenCut/
BoxInst_notok.ipynb          TokenCut_ok.ipynb
CCAM/                        transcam/
CCAM_ok.ipynb                transcam_6485.pth
COCO2YOLO/                   transcam_deeplab.ipynb
Conformer_small_patch16.pth  transcam_instance_ok.ipynb
data_voc/                    transcam_instance_v2_ok.ipynb
DiscoBox/                    transcam_instance_v3_ok.ipynb
DiscoBox_notok.ipynb         transcam_masks_ok.ipynb
externals/                   transcam_ok.ipynb
global-wheat-detection.zip   transcam_train.ipynb
instances_val2017.json       transcam_wheat_ok.ipynb
instance_train_ok.ipynb      transcam_wheat_train.ipynb
isim/                        ultra

In [2]:
import numpy as np
import torch
import os

from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from glob import glob
from tqdm import tqdm
import json

from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch.utils.data import Dataset
import PIL.Image
import os.path
import scipy.misc

In [9]:
# Read names of the each image
image_paths = sorted(glob(os.path.join('wheat/images/train/', "*.jpg")));
label_paths = sorted(glob(os.path.join('wheat/labels/train/', "*.txt")));

In [31]:
img_names = [];
for paths in image_paths:
  img_name = paths.split('/')[-1].split('.')[0];
  img_names.append(img_name);

lbl_names = [];
for paths in label_paths:
  lbl_name = paths.split('/')[-1].split('.')[0];
  lbl_names.append(lbl_name);


counter = 0;
for name in img_names:
  if(name in lbl_names):
    counter = counter+1;
  else:
    image_paths.pop(counter);


In [32]:
print(len(image_paths))
print(len(label_paths))

3374
3374


In [33]:
import shutil


for i in image_paths[1:100]:
  print(i)
  shutil.move(i, 'wheat/images/val/')

wheat/images/train/005b0d8bb.jpg
wheat/images/train/006a994f7.jpg
wheat/images/train/00764ad5d.jpg
wheat/images/train/00b5fefed.jpg
wheat/images/train/00b70a919.jpg
wheat/images/train/00e903abe.jpg
wheat/images/train/00ea5e5ee.jpg
wheat/images/train/010b216d4.jpg
wheat/images/train/010c93b99.jpg
wheat/images/train/010dbcc8f.jpg
wheat/images/train/0114c88aa.jpg
wheat/images/train/01189a3c3.jpg
wheat/images/train/0126b7d11.jpg
wheat/images/train/013669953.jpg
wheat/images/train/01397a84c.jpg
wheat/images/train/013fd7d80.jpg
wheat/images/train/015939012.jpg
wheat/images/train/0172359d2.jpg
wheat/images/train/019d1c76a.jpg
wheat/images/train/01e2c61d9.jpg
wheat/images/train/01f37f1d3.jpg
wheat/images/train/025ea7326.jpg
wheat/images/train/02640d9da.jpg
wheat/images/train/026b6f389.jpg
wheat/images/train/027086635.jpg
wheat/images/train/02992044c.jpg
wheat/images/train/029c79463.jpg
wheat/images/train/029e03e52.jpg
wheat/images/train/02b6f199c.jpg
wheat/images/train/02d662fa8.jpg
wheat/imag

In [35]:
for j in label_paths[1:100]:
  print(j)
  shutil.move(j, 'wheat/labels/val/')

wheat/labels/train/005b0d8bb.txt
wheat/labels/train/006a994f7.txt
wheat/labels/train/00764ad5d.txt
wheat/labels/train/00b5fefed.txt
wheat/labels/train/00b70a919.txt
wheat/labels/train/00e903abe.txt
wheat/labels/train/00ea5e5ee.txt
wheat/labels/train/010b216d4.txt
wheat/labels/train/010c93b99.txt
wheat/labels/train/010dbcc8f.txt
wheat/labels/train/0114c88aa.txt
wheat/labels/train/01189a3c3.txt
wheat/labels/train/0126b7d11.txt
wheat/labels/train/013669953.txt
wheat/labels/train/01397a84c.txt
wheat/labels/train/013fd7d80.txt
wheat/labels/train/015939012.txt
wheat/labels/train/0172359d2.txt
wheat/labels/train/019d1c76a.txt
wheat/labels/train/01e2c61d9.txt
wheat/labels/train/01f37f1d3.txt
wheat/labels/train/025ea7326.txt
wheat/labels/train/02640d9da.txt
wheat/labels/train/026b6f389.txt
wheat/labels/train/027086635.txt
wheat/labels/train/02992044c.txt
wheat/labels/train/029c79463.txt
wheat/labels/train/029e03e52.txt
wheat/labels/train/02b6f199c.txt
wheat/labels/train/02d662fa8.txt
wheat/labe

In [24]:
%ls

AdelaiDet/                   res38_cls.pth
ADL_notok.ipynb              SEAM/
AuxSegNet/                   SESS/
BANA/                        src/
BBAM/                        test_img2.jpg
BBAM_notok.ipynb             test_img.jpg
BESTIE/                      TokenCut/
BoxInst_notok.ipynb          TokenCut_ok.ipynb
CCAM/                        transcam/
CCAM_ok.ipynb                transcam_6485.pth
COCO2YOLO/                   transcam_deeplab.ipynb
Conformer_small_patch16.pth  transcam_instance_ok.ipynb
data_voc/                    transcam_instance_v2_ok.ipynb
DiscoBox/                    transcam_instance_v3_ok.ipynb
DiscoBox_notok.ipynb         transcam_masks_ok.ipynb
externals/                   transcam_ok.ipynb
global-wheat-detection.zip   transcam_train.ipynb
instances_val2017.json       transcam_wheat_ok.ipynb
instance_train_ok.ipynb      transcam_wheat_train.ipynb
isim/                        ultralytics/
isim_notok.ipynb             val_test.ipynb
kaggle.json              